In [43]:
import numpy
# fine tune mt5 on dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from simpletransformers.t5 import T5Model, T5Args
from transformers import pipeline
#import train split
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import torch
import torch.nn as nn
import klib
import os

In [44]:
#load dataset
#English-Hindi code-mixed parallel corpus.csv
df = pd.read_csv('PHNC/English-Hindi code-mixed parallel corpus.csv')
df = df.dropna()
df = df.reset_index(drop=True)
df.head()

,Sentence,English_Translation
0,@someUSER congratulations on you celebrating b...,@some users congratulate you for celebrating B...
1,@LoKarDi_RT uske liye toh bahot kuch karna pad...,"@Lokardi_ rat we should a lot more for that, b..."
2,@slimswamy yehi to hum semjhane ki koshish kar...,"@Slimswami ehi, this is what i'm expecting you..."
3,@DramebaazKudi cake kaha hai ??,@Where is Dramebajakudi where is the cake?
4,@someUSER i'm in hawaii at the moment . home ...,@some user Don't want to come home next friday...


In [45]:
#data cleaning 

df=klib.data_cleaning(df)

Shape of cleaned data: (13737, 2) - Remaining NAs: 0


Dropped rows: 1
     of which 1 duplicates. (Rows (first 150 shown): [1091])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.0 MB (-0.0%)



In [46]:
#split train, val, test
# convert df  so that it can be used by transformers


train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

#print lens
print(len(train))
print(len(val))
print(len(test))

#save train, val, test
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)


8791
2198
2748


In [47]:
df.columns

Index(['sentence', 'english_translation'], dtype='object')

In [48]:
#tokenize
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
tokenizer.add_special_tokens({'additional_special_tokens': ['<sep>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<pad>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['</s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<unk>']})



loading file spiece.model from cache at /home/aparna/.cache/huggingface/hub/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/aparna/.cache/huggingface/hub/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/aparna/.cache/huggingface/hub/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d/tokenizer_config.json
loading configuration file config.json from cache at /home/aparna/.cache/huggingface/hub/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d/config.json
Model config MT5Config {
  "_name_or_path": "google/mt5-small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
 

0

In [49]:
df['sentence']

0        @someUSER congratulations on you celebrating b...
1        @LoKarDi_RT uske liye toh bahot kuch karna pad...
2        @slimswamy yehi to hum semjhane ki koshish kar...
3                          @DramebaazKudi cake kaha hai ??
4        @someUSER i'm in hawaii at the moment .  home ...
                               ...                        
13732    Dr Kumar Vishwas: "Koi deewana kehta hai.. koi...
13733    Me: Aaj kuch toofani karte hai.

Mom: Pani ki ...
13734    Pyar mangi to Jaan dengi,milk mango to kher de...
13735     @imcomplicated__ kaale kaale baal gaal gore gore
13736                            Ye sab aunty'on ke saath?
Name: sentence, Length: 13737, dtype: string

In [74]:
def tokenize_df(df):
    input = tokenizer(df['sentence'], padding=True, truncation=True, return_tensors="pt")
    target = tokenizer(df['english_translation'], padding=True, truncation=True, return_tensors="pt")
    input_ids = input['input_ids']
    attention_mask = input['attention_mask']
    target_ids = target['input_ids']
    target_attention_mask = target['attention_mask']
    decoder_input_ids = target_ids.clone()
    #convert to tensors
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    target_ids = torch.tensor(target_ids)
    target_attention_mask = torch.tensor(target_attention_mask)
   # decoder_input_ids = torch.tensor(decoder_input_ids)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'decoder_input_ids': target_ids,
        #'decoder_input_ids': decoder_input_ids,
        'decoder_attention_mask': target_attention_mask
    }


In [75]:
#tokenize train, val, test
train = load_dataset('csv', data_files='train.csv')
val = load_dataset('csv', data_files='val.csv')
test = load_dataset('csv', data_files='test.csv')
train = train.map(tokenize_df, batched=True, batch_size=len(train),remove_columns=['sentence','english_translation'])
val = val.map(tokenize_df, batched=True, batch_size=len(val),remove_columns=['sentence','english_translation'])
test = test.map(tokenize_df, batched=True, batch_size=len(test),remove_columns=['sentence','english_translation'])


Found cached dataset csv (/home/aparna/.cache/huggingface/datasets/csv/default-5327fb581104ffb6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


100%|██████████| 1/1 [00:00<00:00, 548.20it/s]
Found cached dataset csv (/home/aparna/.cache/huggingface/datasets/csv/default-a2e0709018b1dfe6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


100%|██████████| 1/1 [00:00<00:00, 398.93it/s]
Found cached dataset csv (/home/aparna/.cache/huggingface/datasets/csv/default-f6f7ad1d2e0bf7de/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


100%|██████████| 1/1 [00:00<00:00, 565.65it/s]


/tmp/ipykernel_36803/4255379719.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids)
/tmp/ipykernel_36803/4255379719.py:11: UserWarning: To copy construct from

In [52]:
train

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'target_ids'],
        num_rows: 8791
    })
})

In [53]:
val

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'target_ids'],
        num_rows: 2198
    })
})

In [54]:
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# batch_size = 8
# train_dataloader = DataLoader(

#             train,  # The training samples.

#             sampler = RandomSampler(train), # Select batches randomly

#             batch_size = batch_size # Trains with this batch size.

#         )

# validation_dataloader = DataLoader(

#             val, # The validation samples.

#             sampler = SequentialSampler(val), # Pull out batches sequentially.

#             batch_size = batch_size # Evaluate with this batch size.

#         )

# test_dataloader = DataLoader(


#             test, # The validation samples. 

#             sampler = SequentialSampler(test), # Pull out batches sequentially.

#             batch_size = batch_size # Evaluate with this batch size.

#         )

# #test train data loader
# for batch in train_dataloader:

#     print(batch)

#     break


In [76]:
# finetuen mt5
os.environ["WANDB_DISABLED"] = "true"
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
model.resize_token_embeddings(len(tokenizer))

#training args
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    save_steps=1000,
    eval_steps=1000,
  

)

#trainer
trainer = Seq2SeqTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train["train"],         # training dataset
    eval_dataset=val["train"],             # evaluation dataset
    tokenizer=tokenizer,               # tokenizer
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model), # data collator
)

#train
trainer.train()

#save model
trainer.save_model("./mt5")

loading configuration file config.json from cache at /home/aparna/.cache/huggingface/hub/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d/config.json
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 250112
}

loading weights file pytorch_model.bin from cache at /home/aparna/.cache/huggingface/hub/mod

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`decoder_input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).